In [1]:
import numpy as np
import math
import cv2
import pickle

In [2]:
#Function Definitions

def DoG_on_parasol_bipolars(r, amp):
    a_cent = 1
    a_sur = 0.01
    sig_cent = 4
    sig_sur = sig_cent * 9
    DoG = a_cent * math.e**(-(r**2)/(2*sig_cent**2)) - a_sur * math.e**(-(r**2)/(2*sig_sur**2))
    return DoG*amp

def DoG_on_parasol(r, amp):
    a_cent = 1
    a_sur = 0.375
    sig_cent = 10
    sig_sur = sig_cent * 1.15
    DoG = a_cent * math.e**(-(r**2)/(2*sig_cent**2)) - a_sur * math.e**(-(r**2)/(2*sig_sur**2))
    return DoG*amp

def simulate(cone_array):
    
    #RGC matrices
    on_parasol_responses = np.zeros((64,64))


    #Bipolar matrices
    on_parasol_bipolar_responses = np.zeros((256,256))


    #Cone matrices
    #cone_responses = np.zeros((256,256))
    cone_responses = np.copy(cone_array)
    
    # get cone responses from image, now just given as input
    #cone_responses = image * (0.0001/255.0) #grayscale 0 to 0.0001 
    
    
    
    # get bipolar responses from cone outputs

    # scale up by 11 (10 microns each axis = 5 neurons each way, this means we have 11x11 cones feeding into each bipolar
    # including the cone directly on top of the bipolar)

    i=0
    j=0
    while(i<np.shape(on_parasol_bipolar_responses)[0]):
        while(j<np.shape(on_parasol_bipolar_responses)[1]):

            a_init = 0
            b_init = 0
            a_max = 11
            b_max = 11

            if(j<5):
                b_init=5-j #only can go over to 0th col of cones
            if(i<5):
                a_init=5-i #only can go over to 0th row of cones
            if(j>250):
                b_max=6+(255-j) #only can go over to 255th col of cones
            if(i>250):
                a_max=6+(255-i) #only can go over to 255th row of cones

            a=a_init
            b=b_init
            while(a < a_max):
                while(b < b_max):
                    #scale up and get radii:
                    r = math.sqrt(((a-5)**2) + ((b-5)**2))

                    #get scaled up response:
                    in_resp = cone_responses[i+(a-5),j+(b-5)]
                    resp = DoG_on_parasol_bipolars(r, in_resp)

                    #scale back down by summing:
                    on_parasol_bipolar_responses[i,j] += resp

                    b+=1
                b=b_init
                a+=1

            j+=1
        j=0
        i+=1
    
    #temporal response


    #Since golden paper didn't give any information on the temporal filter (and we are using slightly different
    #temporal modeling with single frame of picture shown then nothing for the rest of the 0.4 s), we will use
    #exponential decay to model the temporal behavior with the initial value starting at the spacial response
    #and the decay value to be 50 (0.02s half life of neuron response strength, close to curve of golden
    #temporal response)

    #response = init_response * e^(decay_val*t)

    decay_val = -50


    #on parasol temporal:

    initial_on_parasol_bipolar_responses = np.copy(on_parasol_bipolar_responses)

    t=0.001 #time steps of 0.001s (400 time periods)
    while t<0.4: #simulate 0.4s like golden paper did
        i=0
        j=0
        while(i<np.shape(on_parasol_bipolar_responses)[0]):
            while(j<np.shape(on_parasol_bipolar_responses)[1]):
                #calculate current time's temporal response value with exponential decay equation, then add it
                #to the overall bipolar response

                init_response = initial_on_parasol_bipolar_responses[i,j]

                response = init_response * math.e**(decay_val*t)

                on_parasol_bipolar_responses[i,j] += response #sum over temporal response for each neuron

                j+=1
            i+=1

        t+=0.001
    
    # get rgc responses from bipolars

    # scale up by 32 from rgc matrix (need 25 microns/axis ~12 neurons each way, means we have 25x25 bipolars feeding each
    # rgc including the bipolar directly on top of the rgc)
    # we need our scaling factor to be a multiple of 4 though (which ignores symmetry), so we will scale by 32x32
    # so that despite assymetry, the value we are losing are essentially 0
    
    i=0
    j=0
    while(i<np.shape(on_parasol_responses)[0]):
        while(j<np.shape(on_parasol_responses)[1]):

            a_init = 0
            b_init = 0
            a_max = 32
            b_max = 32

            if(j*4<16):
                b_init=16-j*4 #only can go over to 0th col of bipolars
            if(i*4<16):
                a_init=16-i*4 #only can go over to 0th row of bipolars
            if(j*4>239):
                b_max=16+(255-j*4) #only can go over to 255th col of bipolars
            if(i*4>239):
                a_max=16+(255-i*4) #only can go over to 255th row of bipolars

            a=a_init
            b=b_init
            while(a < a_max):
                while(b < b_max):

                    #scale up and get radii:
                    r = math.sqrt(((a-16)**2) + ((b-16)**2))

                    #get scaled up response:
                    in_resp = on_parasol_bipolar_responses[i*4+(a-16),j*4+(b-16)]
                    resp = DoG_on_parasol(r, in_resp)

                    #scale baack down by summing:
                    on_parasol_responses[i,j] += resp

                    b+=1
                b=b_init
                a+=1

            j+=1
        j=0
        i+=1
    
    
    #convert response to firing rate


    #go through all rgc responses and convert response to firing rate

    #we'll use an exponential function to get spike rate from rgc response (golden got this from pillow et al 2008)
    #large rgc response is 1.1, so exponential goes from e^0=1 to e^1.2=3.00
    #6 Hz seems to be a large neuron firing rate for an RGC (from other studies), so we'll use this as max_rate
    #exponential function: firing_rate = max_rate*((e^(response)-1)/2.00)
    #subtract 1 from exponential so we have values starting from 0 to 2.00, divide by 2.00 and multiply by max rate so
    #large responses will be about equal to our inputted max firing rate (maybe slightly larger)

    max_rate = 6 #6Hz seems to be a very large RGC firing rate

    #hopefully the learned reconstruction matrix will be able to compensate for any differences we've made in
    #our equations (as long as we use the same equations here and in learning)


    #on parasol rgcs:

    i=0
    j=0
    while(i<np.shape(on_parasol_responses)[0]):
        while(j<np.shape(on_parasol_responses)[1]):
            response = on_parasol_responses[i,j]

            firing_rate = max_rate*((math.e**(response)-1)/2)

            if(firing_rate < 0):
                firing_rate = 0

            on_parasol_responses[i,j] = firing_rate

            j+=1
        j=0
        i+=1
    
    return on_parasol_responses

In [3]:
#path = 'C:/Users/rhyst/Downloads/'
#name = 'test_image.png'
#image = cv2.imread(path+name, cv2.IMREAD_GRAYSCALE)
#cone_array = image * (0.0001/255.0) #grayscale 0 to 0.0001, cones set up to have responses up to 0.0001
#tmp = simulate(cone_array)

In [4]:
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo)
    return dict

imgs = unpickle('D:/ImageNet/Imagenet64_train_part1/train_data_batch_1')
imgs['data'].shape

(128116, 12288)

In [ ]:
#reformatting imagenet images

#from matplotlib import pyplot as plt

test_imgs_tmp = np.copy(imgs['data'][0:500])
test_imgs = np.zeros((500,256*256))
retina_responses = np.zeros((500,64*64))


#tmp = test_imgs_tmp[0].reshape((64,64,3), order='F').transpose(1,0,2)
#gray = cv2.cvtColor(tmp, cv2.COLOR_BGR2GRAY)
#resized = cv2.resize(gray, (256,256), interpolation = cv2.INTER_AREA)
#cone_array = resized * (0.0001/255.0) #grayscale 0 to 0.0001, cones set up to have responses up to 0.0001
#response = simulate(cone_array)

i=0
for row in test_imgs_tmp:
    image = row.reshape((64,64,3), order='F').transpose(1,0,2)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray_image, (256,256), interpolation = cv2.INTER_AREA)
    test_imgs[i] = resized.reshape(256*256)
    
    cone_array = resized * (0.0001/255.0) #grayscale 0 to 0.0001, cones set up to have responses up to 0.0001
    response = simulate(cone_array)
    retina_responses[i] = response.reshape(64*64)
    i+=1



In [ ]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression().fit(retina_responses[0:499], test_imgs[0:8])
test_output = reg.predict(retina_responses[499].reshape((1,64*64)))


In [ ]:
from matplotlib import pyplot as plt
plt.imshow(test_output[0].reshape((256,256)), cmap='gray')
plt.title('reconstruction')
plt.show()

plt.imshow(test_imgs[499].reshape((256,256)), cmap='gray')
plt.title('original')
plt.show()